In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
%%capture
!pip install transformers datasets
!pip install transformers
!pip install transformers[torch]
!pip install transformers[tf-cpu]
!pip install transformers[flax]
!pip install torch
!pip install tensorflow
!pip install evaluate

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import os
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from transformers import AutoTokenizer
import torch
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from transformers import BertForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback, IntervalStrategy, AutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoModelForMaskedLM
from transformers import BertTokenizer, BertModel
from transformers import AdamW, get_linear_schedule_with_warmup

%matplotlib inline
sns.set_theme(style='darkgrid', palette='deep', font='sans-serif', font_scale=1)

## Path

In [ ]:
TRAIN_PATH = Path("/kaggle/input/vitd-violence-inciting-text-detection/train.csv")  
VAL_PATH = Path("/kaggle/input/vitd-violence-inciting-text-detection/dev.csv")
TEST_PATH = Path("/kaggle/input/vitd-violence-inciting-text-detection/test.csv")
# VAL_PRED_PATH = Path("/kaggle/input/vitd-violence-inciting-text-detection/dev-submission-79.4.csv")
TEST_PRED_PATH = Path("/kaggle/input/vitd-violence-inciting-text-detection/99.93test-5_fold-ensemble-test_pred79.35.csv")

# # Training output directory
OUTPUT_DIR = Path("./output/VITD-Models/banglaBert")

MODEL_NAME = 'csebuetnlp/banglabert'

## Imports

In [ ]:
train_df = pd.read_csv(TRAIN_PATH) # train set
val_df = pd.read_csv(VAL_PATH) # validation set

In [ ]:
# add val_data to train
train_df = pd.concat([train_df, val_df], ignore_index=True)

In [ ]:
# # add predicted test data to train
# # val_pred_df = pd.read_csv(VAL_PRED_PATH)
# test_pred_df = pd.read_csv(TEST_PRED_PATH)
# # train_df = pd.concat([train_df, test_pred_df[(test_pred_df["label"] == 1) | (test_pred_df["label"] == 2)]], ignore_index=True)
# train_df = pd.concat([train_df, test_pred_df[(test_pred_df["label"] == 2)]], ignore_index=True)


# # Specify the percentage of rows to randomly select and concatenate
# percentage_to_concat = 0.7  # 10% of the rows

# # Filter test_pred_df to rows where label is 1
# label_1_rows = test_pred_df[test_pred_df["label"] == 1]

# # Calculate the number of rows to select based on the percentage
# num_rows_to_concat = int(len(label_1_rows) * percentage_to_concat)

# # Randomly select rows from label_1_rows
# selected_rows = label_1_rows.sample(n=num_rows_to_concat, random_state=42)

# # Concatenate the selected rows with train_df
# train_df = pd.concat([train_df, selected_rows], ignore_index=True)


## EDA

In [ ]:
print(len(train_df), len(val_df)) 

In [ ]:
train_df.columns

In [ ]:
pd.options.display.max_colwidth = 150
train_df.head() 

In [ ]:
train_df.info()

In [ ]:
# train_df_label_count = train_df.groupby('label', as_index=False)['text'].nunique()
# train_df_label_count.rename(columns={'text':'text_count'}, inplace=True)

plt.figure(figsize=(8, 4))
plt.title("Text vs Label Count Plot")
sns.countplot(data=train_df, x='label')  
plt.show()

In [ ]:
train_df

In [ ]:
# !pip install wordcloud 
# !pip install bnlp-toolkit   

In [ ]:
# import cv2
# import re
# from wordcloud import WordCloud
# from bnlp.corpus import stopwords, punctuations

### Data pre-processing

1. Class imbalance

In [ ]:
# # Count the number of rows where 'label' is 0
# num_zeros = (train_df['label'] == 1).sum()

# print(num_zeros)
# # Determine the number of rows to drop (half of the 'label' 0 count)
# rows_to_drop = num_zeros // 5

# # Filter the DataFrame to get the rows with 'label' equal to 0
# label_zero_rows = train_df[train_df['label'] == 1]

# # Drop half of the rows where 'label' is 0
# train_df = train_df.drop(label_zero_rows.sample(rows_to_drop).index)

# # reset index (recalibrate)
# train_df.reset_index(drop=True, inplace=True)


In [ ]:
# train_df

2. Punctuation and emoji Removal

In [ ]:
#Base and Cleaning
import json
import requests
import emoji
import regex
import re
import string
from collections import Counter


#Natural Language Processing (NLP)
# import spacy
# import gensim
# from spacy.tokenizer import Tokenizer
# from wordcloud import STOPWORDS
# stopwords = set(STOPWORDS)

In [ ]:
!pip install demoji

In [ ]:
import demoji
demoji.download_codes()

In [ ]:
def remove_em(text):
    dem = demoji.findall(text)
    for item in dem.keys():
        text = text.replace(item, '')
    return text
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

In [ ]:

# Apply the function above and get tweets free of emoji's
call_emoji_free = lambda x: remove_em(x)

In [ ]:
def preprocess_text(text):
    text = call_emoji_free(text)
    text = remove_punctuations(text)
    return text

In [ ]:
train_df["text"] = train_df["text"].apply(preprocess_text)

In [ ]:
val_df["text"] = val_df["text"].apply(preprocess_text)

In [ ]:
train_df

In [ ]:
# stopwords.words("bengali")

In [ ]:
# import nltk
# from nltk.corpus import stopwords
# nltk.download('stopwords')
# nltk.download('punkt')


# def remove_bengali_stopwords(text):
#     words = nltk.word_tokenize(text)
#     filtered_words = [word for word in words if word.lower() not in stopwords.words('bengali')]
#     return ' '.join(filtered_words)

# train_df['text'] = train_df['text'].apply(remove_bengali_stopwords)

# Train

In [ ]:
def pandaToList(dataset):
    text_list = []
    label_list = [] 
    for i in range(len(dataset['text'])): 
      text_list.append(dataset['text'][i]) 
      label_list.append(int(dataset['label'][i]))
    return text_list, label_list

train_texts, train_labels =  pandaToList(train_df)
val_texts, val_labels = pandaToList(val_df)  

In [ ]:
# !pip uninstall jaxlibv-y
# !pip install -U jax jaxlib 

In [ ]:
tokenizer_name = MODEL_NAME
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, use_fast=True) 

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

In [ ]:
class ViolenceDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = ViolenceDataset(train_encodings, train_labels)
val_dataset = ViolenceDataset(val_encodings, val_labels)


In [ ]:
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average='macro')
    precision = precision_score(y_true=labels, y_pred=pred, average='macro')
    f1 = f1_score(y_true=labels, y_pred=pred, average='macro')
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [ ]:

# training_args = TrainingArguments(  
#    f"training_with_callbacks",
#    evaluation_strategy = IntervalStrategy.STEPS, # "steps"
#    #output_dir= output_dir, 
#    eval_steps = 250, # Evaluation and Save happens every 250 steps
#    save_total_limit = 5, # Only last 5 models are saved. Older ones are deleted.

#    learning_rate=1e-5,
#    per_device_train_batch_size=16,
#    per_device_eval_batch_size=16,
#    num_train_epochs=50,
#    weight_decay=0.01,
#    metric_for_best_model = 'f1',
#    load_best_model_at_end=True)
 
# model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=3) ## change here for using another pretrained model

# # Initialize the optimizer
# optimizer = AdamW(model.parameters(), lr=training_args.learning_rate)

# # Get the total number of training steps
# total_steps = len(train_dataset) * training_args.num_train_epochs

# # Create a learning rate scheduler
# scheduler = get_linear_schedule_with_warmup(
#     optimizer=optimizer,
#     num_warmup_steps=int(total_steps * 0.1),  # Warmup steps as a percentage of total steps
#     num_training_steps=total_steps
# )

# trainer = Trainer(
#     model=model,                         # the instantiated Transformers model to be trained
#     args=training_args,                  # training arguments, defined above
#     train_dataset=train_dataset,         # training dataset
#     eval_dataset=val_dataset,            # evaluation dataset
#     compute_metrics=compute_metrics,
#     tokenizer=tokenizer, 
#     callbacks = [EarlyStoppingCallback(early_stopping_patience=7)]
# )

In [ ]:
from sklearn.model_selection import StratifiedKFold

training_args = TrainingArguments(  
   f"training_with_callbacks",
   evaluation_strategy = IntervalStrategy.STEPS, # "steps"
   #output_dir= output_dir, 
   eval_steps = 250, # Evaluation and Save happens every 250 steps
   save_total_limit = 5, # Only last 5 models are saved. Older ones are deleted.

   learning_rate=1e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=50,
   weight_decay=0.01,
   metric_for_best_model = 'f1',
   load_best_model_at_end=True)

model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=3)

# Number of folds for k-fold cross-validation
num_folds = 5

# Initialize StratifiedKFold
kf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

# Lists to store evaluation results
all_train_metrics = []
all_val_metrics = []

# Iterate through each fold
for fold, (train_idx, val_idx) in enumerate(kf.split(train_df['text'], train_df['label'])):
    print(f"Fold {fold + 1}:")

    # Create train and validation datasets for this fold
    train_fold_df = train_df.iloc[train_idx].reset_index(drop=True)
    val_fold_df = train_df.iloc[val_idx].reset_index(drop=True)
    
    train_fold_texts, train_fold_labels =  pandaToList(train_fold_df)
    val_fold_texts, val_fold_labels = pandaToList(val_fold_df)  
    
    train_fold_encodings = tokenizer(train_fold_texts, truncation=True, padding=True)
    val_fold_encodings = tokenizer(val_fold_texts, truncation=True, padding=True)
    
    train_fold = ViolenceDataset(train_fold_encodings, train_fold_labels)
    val_fold = ViolenceDataset(val_fold_encodings, val_fold_labels)

    # Initialize the model and optimizer
    optimizer = AdamW(model.parameters(), lr=training_args.learning_rate)
    total_steps = len(train_fold) * training_args.num_train_epochs
    scheduler = get_linear_schedule_with_warmup(
        optimizer=optimizer,
        num_warmup_steps=int(total_steps * 0.1),
        num_training_steps=total_steps
    )

    # Initialize the trainer for this fold
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_fold,
        eval_dataset=val_fold,
        compute_metrics=compute_metrics,
        tokenizer=tokenizer,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
    )

    # Train the model for this fold
    trainer.train()

    # Evaluate the model on train and validation datasets
    train_metrics = trainer.evaluate(train_fold)
    val_metrics = trainer.evaluate(val_fold)
    all_train_metrics.append(train_metrics)
    all_val_metrics.append(val_metrics)

# Print evaluation results for each fold
for fold in range(num_folds):
    print(f"Fold {fold + 1}:")
    print("Train Metrics:", all_train_metrics[fold])
    print("Validation Metrics:", all_val_metrics[fold])


In [ ]:
# wandb-api: 
# trainer.train()  

### save the model

In [ ]:
# Check if the OUTPUT_DIR exists, if not, create it
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# saving the model to drive.
trainer.save_model(f"{OUTPUT_DIR}") 

In [ ]:
trainer.predict(val_dataset)  

In [ ]:
# from IPython.display import FileLink
  
# # Assuming you have saved the pytorch_models.bin in /kaggle/working directory
# file_path = '/kaggle/working/output/VITD-Models/banglaBert/pytorch_model.bin'

# display(FileLink(file_path)) 

## generate submission file

In [ ]:
# Load the trained model and tokenizer from the saved checkpoint
OUTPUT_DIR = Path("/kaggle/working/output/VITD-Models/banglaBert")
SUBMISSION_FILE_DIR = Path("/kaggle/working/output/VITD-Predictions")

model = AutoModelForSequenceClassification.from_pretrained(OUTPUT_DIR, num_labels=3)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

In [ ]:
# Assuming you have new data in a pandas DataFrame called 'test_df' with a 'text' column containing the input text
test_df = pd.read_csv(TEST_PATH)  # Replace 'test.csv' with the filename of your test dataset

In [ ]:
test_df["text"] = test_df["text"].apply(preprocess_text)

In [ ]:
def pandaToList(dataset):
    text_list = []
    for i in range(len(dataset['text'])):
        text_list.append(dataset['text'][i])
    return text_list

class ViolenceTestDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

In [ ]:
test_texts = pandaToList(test_df)   

# Tokenize the test data
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

# Create a test dataset
test_dataset = ViolenceTestDataset(test_encodings) 

# Set up DataLoader for batching
batch_size = 16
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

In [ ]:
# Put the model in evaluation mode
model.eval() 

In [ ]:
# List to store the predictions
predictions = []

# Perform inference on the test dataset
with torch.no_grad():
    for batch in test_dataloader:
        inputs = {key: val.to(model.device) for key, val in batch.items()} 
        outputs = model(**inputs)
        logits = outputs.logits
        predictions.extend(torch.argmax(logits, dim=1).tolist())

In [ ]:
# Convert the predictions to a list of labels (assuming your labels are 0, 1, and 2)
predicted_labels = [int(pred) for pred in predictions]

In [ ]:
print(classification_report(test_df['label'], predicted_labels,digits=4))

In [ ]:
test_df['label'] = predicted_labels

In [ ]:
# Check if the OUTPUT_DIR exists, if not, create it
if not os.path.exists(SUBMISSION_FILE_DIR):
    os.makedirs(SUBMISSION_FILE_DIR)

# save the prediction file to drive
test_df.to_csv(f"{SUBMISSION_FILE_DIR}/submission-emoji_punctuation-test_pred79.28-0_0-1_7-2_1.csv", index=False)

In [ ]:
test_df